# Get Vacancy by Radius

### 12/31/17

- get vacantOSMNX_1516KenhillAve1800_123117

### 12/30/17

- get vacantOSMNX_1516KenhillAve1200_123017

- get vacantOSMNX_1516KenhillAve1300_123017

### 12/29/17

- get vacantOSMNX_1516KenhillAve1000_122917

- get vacantOSMNX_1516KenhillAve1100_122917

### 12/28/17

- delete Set Edge

- get vacantOSMNX_1516KenhillAve800_122817

- get vacantOSMNX_1516KenhillAve900_122917

In [ ]:
# get clean vacant
import csv
import pandas as pd
import time
vacant_Baltimore = pd.read_csv('CleanDataBase/vacantBaltimore_171117_8000.csv',sep='\t') # create header later
vacant_Baltimore = vacant_Baltimore.drop("Unnamed: 0", 1)

# clean street name
vacant_Baltimore.loc[:,"street"] = vacant_Baltimore.loc[:,"street"].str[1:-1]
print "number of Vacancy in db25:", len(vacant_Baltimore)

In [ ]:
# get osmnx GEOdataframe
import osmnx as ox
address = '1516 Kenhill Ave, Baltimore, MD'
radius = 1800
gdf = ox.buildings_from_address(address, distance=radius)

print "Address:", address
print "radius", radius
print "number of buildings from osmnx:", len(gdf)

In [ ]:
## check iff street is not null and building type is yes
result = pd.concat([gdf["addr:street"].isnull() == False, gdf["building"] == "yes" ], axis=1)
result["sum"] = result["addr:street"] & result["building"]
street_gdf = gdf[result["sum"]]

print "number of buildings with street name is not null:", len(street_gdf)

In [ ]:
def vacant_generator(i,j):
    
    while i < len(street_gdf.index):
        while j < len(vacant_Baltimore.index):
            if (street_gdf['addr:street'][street_gdf.index[i]] == vacant_Baltimore['street'][vacant_Baltimore.index[j]] and 
                street_gdf['addr:housenumber'][street_gdf.index[i]] == vacant_Baltimore['housenumber'][vacant_Baltimore.index[j]]):
                
                yield street_gdf.index[i],vacant_Baltimore.index[j], street_gdf['addr:street'][street_gdf.index[i]], street_gdf['addr:housenumber'][street_gdf.index[i]]
            yield street_gdf.index[i],vacant_Baltimore.index[j],None,None
            j += 1
        
        if i%1000==0:
            print i
            
        i += 1
        j = 0
        
                #m.gdf.loc[i,'housetype'] = 2
            #elif m.gdf['housetype'][i] == 2:
                #m.gdf.loc[i,'housetype'] = 1

In [ ]:
vacantOSMNX = []
vacantdb25 = []
s = time.time()
for i,j,street,housenumber in vacant_generator(0,0):
    if street != None:
        print street, housenumber, i, j
        vacantOSMNX.append(i)
        vacantdb25.append(j)
print "get", len(vacantOSMNX), "houses using", time.time() - s, "s"

In [ ]:
print (time.time() - s)/60 , "mins"

In [ ]:
len(vacantdb25)

In [ ]:
# write data into csv
with open('VacantSet-OSMNX/vacantOSMNX_1516KenhillAve1800_123117', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(vacantOSMNX)
    
"""
# Read data from csv
with open('vacantOSMNX_Baltimore_171117', 'rb') as f:
    reader = csv.reader(f)
    your_list = map(int,list(reader)[0])
""" ;

# Record